In [1]:
import torch
from transformers import SegformerFeatureExtractor, SegformerForSemanticSegmentation, SegformerConfig
backbone = SegformerForSemanticSegmentation.from_pretrained("nvidia/segformer-b0-finetuned-ade-512-512")
backbone.decode_head.classifier = torch.nn.Identity()


/home/wg25r/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/segformer-b0-finetuned-ade-512-512 and are newly initialized: ['segformer.encoder.test.bias', 'segformer.encoder.test.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
backbone.eval()
import os

In [3]:
import torch
frames = os.listdir('/mnt/fastdata/dataset/baseline/office/input')[::5]
import tqdm
features = torch.zeros((len(frames), 256, 128, 128))
len(frames)

410

In [4]:
from PIL import Image
from transformers import AutoImageProcessor
image_processor = AutoImageProcessor.from_pretrained("nvidia/segformer-b0-finetuned-ade-512-512")
backbone = backbone.cuda()

In [5]:
for i in tqdm.tqdm(range(len(frames))):
    with torch.no_grad():
        frame = frames[i]
        img = Image.open(f'/mnt/fastdata/dataset/baseline/office/input/{frame}')
        img = image_processor(img, return_tensors="pt")
        res = backbone(img.pixel_values.cuda()).logits.cpu()
        features[i] = res   

100%|██████████| 410/410 [00:11<00:00, 36.47it/s]


In [15]:
mean = features[0]

sim = torch.nn.CosineSimilarity(dim=1)(mean, features)


In [16]:
difference = 1 - sim

In [17]:
difference.shape

torch.Size([410, 128, 128])

In [18]:
difference = difference.cpu().numpy()
# difference = difference.mean(1)


In [22]:
# save difference into a mp4 video 
import cv2
import numpy as np
import os 
import tqdm

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video = cv2.VideoWriter('difference.mp4', fourcc, 10, (128 * 2, 128))
for i in tqdm.tqdm(range(len(difference))):
    mask = (difference[i] > 0.3) * 255 
    mask = cv2.applyColorMap(mask.astype(np.uint8), cv2.COLORMAP_JET)
    raw = cv2.imread(f'/mnt/fastdata/dataset/baseline/office/input/{frames[i]}')
    raw = cv2.resize(raw, (128, 128))
    frame = np.concatenate((raw, mask), axis=1)
    video.write(frame)
video.release() 

100%|██████████| 410/410 [00:00<00:00, 934.02it/s]


In [9]:
# kmeans (kind of similar to mog) then chose the most common cluster as the background (even gpt knows this)